<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Turning_Website_Content_Into_Password_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Turning Website Content into Password Gold__*

Many times, security comes down to one thing: User passwords. It's sad but true. Making things worse, when it comes to web applications, especially custom ones, it's all too common to discover that they don't lock users out of their accounts after a certain number of failed authentication attempts. In other instances, they don't enforce strong passwords. In these cases, an online password guessing session like the one in the last chapter might be just the ticket to gain access to the site.

The trick to online password guessing is getting the right word list. You can't test 10 million passwords if you're in a hurry, so you need to be able to create a word list targeted to the site in question. Of course, there are scripts in Kali Linux that crawl a website and generate a word list based on site content. But if you've already used Burp to scan the site, why send more traffic just to generate a word list? Plus, those scripts usually have a ton of command line arguments to remember. If you're anything like us, you've already memorized enough command line arguments to impress your friends, so let's make Burp do the heavy lifting.

Open __bhp_wordlist.py__ and knock out this code:

In [ ]:
from burp import IBurpExtender
from burp import IContextMenuFactory

from java.util import ArrayList
from java.swing import JMenuItem

from datetime import datetime
from HTMLParser import HTMLParser

import re

class TagStripper(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.page_text = []

    def handle_data(self, data):
        self.page_text.append(data) #[1]

    def handle_comment(self, data):
        self.page_text.append(data) #[2]

    def strip(self, html):
        self.feed(html)
        return " ".join(self.page_text) #[3]

class BurpExtender(IBurpExtender, IContextMenuFactory):
    def registerExtenderCallbacks(self, callbacks):
        self._callbacks = callbacks
        self._helpers = callbacks.getHelpers()
        self.context = None
        self.hosts = set()

        # Start with something we know is common
        self.wordlist = set(["password"]) #[4]

        # We set up our extension
        callbacks.setExtensionName("BHP Wordlist")
        callbacks.registerContextMenuFactory(self)

        return

    def createMenuItems(self, context_menu):
        self.context = context_menu
        menu_list = ArrayList()
        menu_list.add(JMenuItem("Create Wordlist", actionPerformed=self.wordlist_menu))

        return menu_list

The code in this listing should be pretty familiar by now. We start by importing the required modules. A helper __TagStripper__ class will allow us to strip the HTML tags out of the HTTP responses we process later on. Its __handle_data__ method store the page text __[1]__ in a member variable. We also define the __handle_comment__ method because we want to add the words stored in developer comments to the password list as well. Under the covers, __handle_comment__ just calls __handle_data__ __[2]__ (in case we want to change how we process page text down the road).

The __strip__ method feeds HTML code to the base class, __HTMLParser__, and returns the resulting page text __[3]__, which will come in handy later. The rest is almost exactly the same as the start of the __bhp_bing.py__ script we just finished. Once again, the goal is to create a context menu item in the Burp UI. The only thing new here is that we store our word list in a __set__, which ensures that we don't introduce duplicate words as we go. We initialize the __set__ with everyone's favorite password, __password__ __[4]__, just to make sure it ends up in our final list.

Now let's add the logic to take the selected HTTP traffic from Burp and turn it into a base word list:

In [ ]:
class BurpExtender(IBurpExtender, IContextMenuFactory):
    def wordlist_menu(self, event):
         # Grab the details of what the user clicked
         http_traffic = self.context.getSelectedMessages()

         for traffic in http_traffic:
             http_service = traffic.getHttpService()
             host = http_service.getHost()
             self.hosts.add(host) #[1]

             http_response = traffic.getResponse()
             if http_response:
                 self.get_words(http_response) #[2]

        self.display_wordlist()
        return

    def get_words(self, http_response):
        headers, body = http_response.tostring().split("\r\n\r\n", 1)

        # Skip non-text responses
        if headers.lower().find("content-type: text") == -1: #[3]
            return

        tag_stripper = TagStripper()
        page_text = tag-stripper.strip(body) #[4]

        words = re.findall("[a-zA-Z]\w{2,}", page_text) #[5]

        for word in words:
            # Filter out long strings
            if len(word) <= 12:
                self.wordlist.add(word.lower()) #[6]

        return

Our first order of business is to define the __wordlist_menu__ method, which handles menu clicks. It saves the name of the responding host __[1]__ for later and then retrieves the HTTP response and feeds it to the __get_words__ method __[2]__.
From there, __get_words__ checks the response header to make sure we're processing text-based responses only __[3]__. The __TagStripper__ class __[4]__ strips the HTML code from the rest of the page text. We use a regular expression to find all words starting with an alphabetic character and two or more "word" characters as specified with the _\w{2,}_ regular expression __[5]__. We save the words that match this pattern to the __wordlist__ in lowrcase __[6]__.

Now let's polish the script by giving it the ability to mangle and display the capture word list:

In [ ]:
class BurpExtender(IBurpExtender, IContextMenuFactory):
    def mangle(self, word):
        year = datetime.now().year
        suffixes = ['', '1', '!', year] #[1]
        mangled = []

        for password in (word, word.capitalize()):
            for suffix in suffixes:
                mangled.append("%s%s" % (password, suffix)) #[2]

        return mangled

    def display_wordlist(self):
        print("#!comment: BHP Wordlist for site(s) %s" % ", ".join(self.hosts)) #[3]

        for word in sorted(self.wordlist):
            for password in self.mangle(word):
                print(password)

        return

Very nice! The __mangle__ method takes a base word and turns it into a number of password guesses based on some common password creation strategies. In this simple example, we create a list of suffixes to tack on the end of the base word, including the current year __[1]__. Next, we loop through each suffix and add it to the base word __[2]__ to create a unique password attempt. We do another loop with a capitalized version of the base word for good measure. In the __display_wordlist__ method, we print a "John The Ripper" style comment __[3]__ to remind us which sites we used to generate this word list. Then we mangle each base word and print the results. Time to take this baby for a spin.

# *__Kicking the Tires__*

Click the __Extender__ tab in Burp, click the __Add__ button, and then use the same procedure we used for our previous extensions to get the Wordlist extension working. In the Dashboard tab, select __New live task__, as shown in _Figure 6-12_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Starting_A_Live_Passive_Scan_With_Burp.png?raw=true" alt="Starting A Live Passive Scan With Burp" width="70%">
<p style="text-align:center"><strong><em>Figure 6-12</strong></em></p>
</div>

When the dialog appears, choose __Add all links observed in traffic__, as shown in _Figure 6-12_, and click __OK__.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Configuring_The_Live_Passive_Scan_With_Burp.png?raw=true" alt="Configuring The Live Passive Scan With Burp" width="70%">
<p style="text-align:center"><strong><em>Figure 6-13</strong></em></p>
</div>

After you've configured the scan, browse to _http://testphp.vulnweb.com/_ to run it. Once Burp has visited all the links on the target site, select all the requests in the top-right pane of the __Target__ tab, right-click them to bring up the context menu, and select __Create Wordlist__, as shown in _Figure 6-14_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Sending_The_Requests_To_The_BHP_Wordlist_Extension.png?raw=true" alt="Sending The Requests To The BHP Wordlist Extension" width="70%">
<p style="text-align:center"><strong><em>Figure 6-14</strong></em></p>
</div>

Now check the Output tab of the extension. In practice, we'd save its output to a file, but for demonstration purposes we display the word list in Burp, as shown in _Figure 6-15_.

You can now feed this list back into Burp Intruder to perform the actual password-guessing attack.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/A_Password_List_Based_On_Content_From_The_Target_Website.png?raw=true" alt="A Password List Based On Content From The Target Website" width="70%">
<p style="text-align:center"><strong><em>Figure 6-15</strong></em></p>
</div>

We've now demonstrated a small subset of the Burp API by generating our own attack payloads, as well as building extensions that interact with the Burp UI. During a penetration test, you'll often encounter specific problems or automation needs, and the Burp Extender API provides an excellent interface to code your way out of a corner, or at least save you from having to continually copy and paste captured data from Burp to another tool.